# Mini Project 5-5 Explore Hypothesis Testing

## Introduction

You work for an environmental think tank called Repair Our Air (ROA). ROA is formulating policy recommendations to improve the air quality in America, using the Environmental Protection Agency's Air Quality Index (AQI) to guide their decision making. An AQI value close to 0 signals "little to no" public health concern, while higher values are associated with increased risk to public health. 

They've tasked you with leveraging AQI data to help them prioritize their strategy for improving air quality in America.

ROA is considering the following decisions. For each, construct a hypothesis test and an accompanying visualization, using your results of that test to make a recommendation:

1. ROA is considering a metropolitan-focused approach. Within California, they want to know if the mean AQI in Los Angeles County is statistically different from the rest of California.
2. With limited resources, ROA has to choose between New York and Ohio for their next regional office. Does New York have a lower AQI than Ohio?
3. A new policy will affect those states with a mean AQI of 10 or greater. Will Michigan be affected by this new policy?

**Notes:**
1. For your analysis, you'll default to a 5% level of significance.
2. Throughout the lab, for two-sample t-tests, use Welch's t-test (i.e., setting the `equal_var` parameter to `False` in `scipy.stats.ttest_ind()`). This will account for the possibly unequal variances between the two groups in the comparison.

## Step 1: Imports

To proceed with your analysis, import `pandas` and `numpy`. To conduct your hypothesis testing, import `stats` from `scipy`.

#### Import Packages

In [2]:
# Import relevant packages
import pandas as pd
from scipy import stats

You are also provided with a dataset with national Air Quality Index (AQI) measurements by state over time for this analysis. `Pandas` was used to import the file `c4_epa_air_quality.csv` as a dataframe named `aqi`. As shown in this cell, the dataset has been automatically loaded in for you. You do not need to download the .csv file, or provide more code, in order to access the dataset and proceed with this lab. Please continue with this activity by completing the following instructions.

**Note:** For purposes of your analysis, you can assume this data is randomly sampled from a larger population.

#### Load Dataset

In [3]:
# IMPORT YOUR DATA
c4_epa_air_quality = pd.read_csv("c4_epa_air_quality.csv")

## Step 2: Data Exploration

### Before proceeding to your deliverables, explore your datasets.

Use the following space to surface descriptive statistics about your data. In particular, explore whether you believe the research questions you were given are readily answerable with this data.

In [4]:
# Use head() to show a sample of data
c4_epa_air_quality.head(15)

,Unnamed: 0,date_local,state_name,county_name,city_name,local_site_name,parameter_name,units_of_measure,arithmetic_mean,aqi
0,0,2018-01-01,Arizona,Maricopa,Buckeye,BUCKEYE,Carbon monoxide,Parts per million,0.473684,7
1,1,2018-01-01,Ohio,Belmont,Shadyside,Shadyside,Carbon monoxide,Parts per million,0.263158,5
2,2,2018-01-01,Wyoming,Teton,Not in a city,Yellowstone National Park - Old Faithful Snow ...,Carbon monoxide,Parts per million,0.111111,2
3,3,2018-01-01,Pennsylvania,Philadelphia,Philadelphia,North East Waste (NEW),Carbon monoxide,Parts per million,0.300000,3
4,4,2018-01-01,Iowa,Polk,Des Moines,CARPENTER,Carbon monoxide,Parts per million,0.215789,3
5,5,2018-01-01,Hawaii,Honolulu,Not in a city,Kapolei,Carbon monoxide,Parts per million,0.994737,14
6,6,2018-01-01,Hawaii,Honolulu,Not in a city,Kapolei,Carbon monoxide,Parts per million,0.200000,2
7,7,2018-01-01,Pennsylvania,Erie,Erie,NaN,Carbon monoxide,Parts per million,0.200000,2
8,8,2018-01-01,Hawaii,Honolulu,Honolulu,Honolulu,Carbon monoxide,Parts per million,0.400000,5
9,9,2018-01-01,Colorado,Larimer,Fort Collins,Fort Collins - CSU - S. Mason,Carbon monoxide,Parts per million,0.300000,6


In [5]:
# check varibles
c4_epa_air_quality.size

2600

In [8]:
# Use describe() to summarize AQI
aqi_summary = c4_epa_air_quality["aqi"].describe()
aqi_summary

count    260.000000
mean       6.757692
std        7.061707
min        0.000000
25%        2.000000
50%        5.000000
75%        9.000000
max       50.000000
Name: aqi, dtype: float64

In [9]:
# For a more thorough examination of observations by state use values_counts()
state_counts = c4_epa_air_quality["state_name"].value_counts()
state_counts


state_name
California              66
Arizona                 14
Ohio                    12
Florida                 12
Texas                   10
Pennsylvania            10
New York                10
Colorado                 9
Michigan                 9
Minnesota                7
New Jersey               6
Indiana                  5
Nevada                   4
Maryland                 4
Massachusetts            4
Oklahoma                 4
North Carolina           4
Virginia                 4
Connecticut              4
Wyoming                  3
Utah                     3
Hawaii                   3
Illinois                 3
Missouri                 3
Iowa                     3
Kentucky                 3
Vermont                  3
Rhode Island             2
Alaska                   2
Georgia                  2
Washington               2
Idaho                    2
Nebraska                 2
Montana                  2
Tennessee                2
Maine                    2
New Mexico       

#### **Question 1: From the preceding data exploration, what do you recognize?**

A: California has the most occurance

## Step 3. Statistical Tests

Before you proceed, recall the following steps for conducting hypothesis testing:

1. Formulate the null hypothesis and the alternative hypothesis.<br>
2. Set the significance level.<br>
3. Determine the appropriate test procedure.<br>
4. Compute the p-value.<br>
5. Draw your conclusion.

### Hypothesis 1: ROA is considering a metropolitan-focused approach. Within California, they want to know if the mean AQI in Los Angeles County is statistically different from the rest of California.

Before proceeding with your analysis, it will be helpful to subset the data for your comparison.

In [10]:
# Create dataframes for each sample being compared in your test
la_aqi = c4_epa_air_quality[
    (c4_epa_air_quality["state_name"] == "California") & 
    (c4_epa_air_quality["county_name"] == "Los Angeles")
]["aqi"]
ca_other_aqi = c4_epa_air_quality[
    (c4_epa_air_quality["state_name"] == "California") & 
    (c4_epa_air_quality["county_name"] != "Los Angeles")
]["aqi"]


In [11]:
# Check head
print("Los Angeles County AQI Data:")
print(la_aqi.head())
print("\nRest of California AQI Data:")
print(ca_other_aqi.head())

Los Angeles County AQI Data:
33      7
42      7
61     16
76     17
109    17
Name: aqi, dtype: int64

Rest of California AQI Data:
16    11
18    16
26    13
27    15
34    15
Name: aqi, dtype: int64


#### Formulate your hypothesis:

**Formulate your null and alternative hypotheses:**

*   $H_0$: There is no difference in the mean AQI between Los Angeles County and the rest of California.
*   $H_A$: There is a difference in the mean AQI between Los Angeles County and the rest of California.


#### Set the significance level:

In [ ]:
# For this analysis, the significance level is 5%
significance_level = 0.05

#### Determine the appropriate test procedure:

Here, you are comparing the sample means between two independent samples. Therefore, you will utilize a **two-sample  𝑡-test**.

#### Compute the P-value

In [13]:
# Compute your p-value here
stats.ttest_ind(a=la_aqi, b=ca_other_aqi, equal_var=False)

TtestResult(statistic=np.float64(2.1107010796372014), pvalue=np.float64(0.049839056842410995), df=np.float64(17.08246830361151))

#### **Question 2. What is your P-value for hypothesis 1, and what does this indicate for your null hypothesis?**

In [ ]:
# Extracting pvalue and make the test
p_value = 0.0498



A: reject null hypothesis

### Hypothesis 2: With limited resources, ROA has to choose between New York and Ohio for their next regional office. Does New York have a lower AQI than Ohio?

Before proceeding with your analysis, it will be helpful to subset the data for your comparison.

In [14]:
# Create dataframes for each sample being compared in your test
ny_aqi = c4_epa_air_quality[c4_epa_air_quality["state_name"] == "New York"]["aqi"]

# Create a dataframe for AQI in Ohio
ohio_aqi = c4_epa_air_quality[c4_epa_air_quality["state_name"] == "Ohio"]["aqi"]

In [17]:
# Check head
ny_aqi.head()
ohio_aqi.head()


1     5
12    3
22    6
51    3
59    3
Name: aqi, dtype: int64

**Formulate your null and alternative hypotheses:**

*   $H_0$: The mean AQI of New York is greater than or equal to that of Ohio.
*   $H_A$: The mean AQI of New York is **below** that of Ohio.


#### Significance Level (remains at 5%)

#### Determine the appropriate test procedure:

Here, you are comparing the sample means between two independent samples in one direction. Therefore, you will utilize a **two-sample  𝑡-test**.

#### Compute the P-value

In [31]:
# Compute your p-value here
statistics, p_value = stats.ttest_ind(a=ny_aqi, b=ohio_aqi, equal_var=False)
print ("pvalue:",p_value)

pvalue: 0.06089300538386936


#### **Question 3. What is your P-value for hypothesis 2, and what does this indicate for your null hypothesis?**

In [32]:
# Your code here.

if p_value < 0.05:
    print('p_value < 0.05, Reject Ho.')          # Ha: There is a difference in the mean
else:
    print('p_value > 0.05, Fail to reject Ho.')

p_value > 0.05, Fail to reject Ho.


A: Fail to reject null hypothesis

###  Hypothesis 3: A new policy will affect those states with a mean AQI of 10 or greater. Will Michigan be affected by this new policy?

Before proceeding with your analysis, it will be helpful to subset the data for your comparison.

In [34]:
# Create dataframes for each sample being compared in your test
michigan_aqi = c4_epa_air_quality[c4_epa_air_quality["state_name"] == "Michigan"]["aqi"]
michigan_mean_aqi = michigan_aqi.mean()
policy_threshold = 10



**Formulate your null and alternative hypotheses here:**

*   $H_0$: The mean AQI of Michigan is less than or equal to 10.
*   $H_A$: The mean AQI of Michigan is greater than 10.


#### Significance Level (remains at 5%)

#### Determine the appropriate test procedure:

Here, you are comparing one sample mean relative to a particular value in one direction. Therefore, you will utilize a **one-sample  𝑡-test**. 

#### Compute the P-value

In [35]:
# Compute your p-value here
statistics, p_value = stats.ttest_1samp(michigan_aqi, policy_threshold)
p_value

np.float64(0.12011896137197813)

#### **Question 4. What is your P-value for hypothesis 3, and what does this indicate for your null hypothesis?**

A: p_value = 0.1201, fail to reject null hypothesis

## Step 4. Results and Evaluation

Now that you've completed your statistical tests, you can consider your hypotheses and the results you gathered.

#### **Question 5. Did your results show that the AQI in Los Angeles County was statistically different from the rest of California?**

A: Yes, they are different

#### **Question 6. Did New York or Ohio have a lower AQI?**

A: Ohio has a lower AQI

#### **Question 7: Will Michigan be affected by the new policy impacting states with a mean AQI of 10 or greater?**



A: Michigan will not be affected

# Conclusion

**What are key takeaways from this project?**

A: to conduct t-test and calculate p-value

**What would you consider presenting to your manager as part of your findings?**

A: the visualizaions of each result


**What would you convey to external readers?**

A: LA is different form other counties in CA, NYC has greater AQI than Ohio, Michigan will not be affected by the new policy
